In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
import sys
from geopy import distance
from datetime import datetime

In [2]:
import sys
dir_path = os.path.join(os.getcwd(), '../', 'scripts/')
sys.path.insert(1, dir_path)
from resample import return_resampled_df

In [3]:
def format_data(acc_df, label_df):
    if 'Latitude' not in acc_df:
        acc_df.rename(columns = {'Lat': 'Latitude'}, inplace = True) 
        acc_df.rename(columns = {'Lng': 'Longitude'}, inplace = True)
        
    time = label_df['Time']
    if label_df['Time'].dtype != float:
        time = [datetime.strptime(x, '%Y-%m-%d %H:%M:%S:%f') for x in time]
    time = pd.Series(time)
    time = time.apply(lambda x: x.timestamp())
        
    time_ = acc_df['Time']
    if acc_df['Time'].dtype != float:
        time_ = [datetime.strptime(x, '%Y-%m-%d %H:%M:%S:%f') for x in time_]
        time_ = pd.Series(time_)
        time_ = time_.apply(lambda x: x.timestamp())

    time = time - time_[0]
    label_df['Time'] = time
    return acc_df, label_df

In [14]:
def get_merged_label_data(date, data_nums, type='roughness'):
    data = pd.DataFrame()
    last_time = 0
    for data_num in data_nums:
        label_path = os.path.join(os.getcwd(), '../data/labels/'+date+'/'+'auto_'+str(data_num)+'_labels.csv')
        acc_path = os.path.join(os.getcwd(), '../data/acc/'+date+'/pickledData/pickled'+str(data_num)+'.pkl')
        cur_data = return_resampled_df(0, label_path=label_path, acc_path=acc_path)
        time_arr = cur_data['Time'].to_numpy()
        time_arr = time_arr + last_time
        cur_data['Time'] = time_arr
        last_time = time_arr[-1]
        # print(cur_data)
        if not cur_data.empty:
            data = pd.concat([data, cur_data])
        data = data.reset_index(drop=True)
    event_vals = [0, 5, 6, 7, 8]
    if type == 'roughness':
        final_data = data.loc[~data['Label'].isin(event_vals)]
    elif type == 'event':
        data.loc[~data['Label'].isin(event_vals), 'Label'] = 0
        final_data = data
    final_data = final_data.reset_index(drop=True)
    return final_data

In [15]:
date = '29March2024'
data_nums = [1, 3, 5, 8]

In [16]:
merged_data = get_merged_label_data(date, data_nums)
print(merged_data)

Individual time taken by pre-process: 0.01 seconds for data size: 10982
Individual time taken by resample: 0.81 seconds for data size: 10982
Individual time taken by pre-process: 0.0 seconds for data size: 8929
Individual time taken by resample: 0.64 seconds for data size: 8929
Individual time taken by pre-process: 0.0 seconds for data size: 1414
Individual time taken by resample: 0.1 seconds for data size: 1414
Individual time taken by pre-process: 0.0 seconds for data size: 8350
Individual time taken by resample: 0.6 seconds for data size: 8350
          x_acc      y_acc     z_acc         Time  Velocity  Prev_Velocity  \
0      9.663713  -0.851546 -0.629611     0.000000  0.099691            0.0   
1      9.684225  -0.853255 -0.915493     0.020002  0.099691            0.0   
2      9.713227  -0.849977 -0.998117     0.040004  0.099691            0.0   
3      9.738841  -0.844847 -0.956117     0.060006  0.099691            0.0   
4      9.713315  -0.848037 -0.975262     0.080007  0.0996

In [19]:
filename = ''
for data_num in data_nums:
    filename += str(data_num)
filename = str(date)+'_'+filename+'.csv'
merged_data.to_csv('../data/test_data/'+filename)